In [28]:
# Machine Learning
# Currently set for breast cancer data
# make sure the data is classified numerically (e.g. 10 classifications = output column with 0,1,...,8,9)
# Three algorithms shown here
# 1. Linear Regression
# 2. Logistic Regression
# 3. Neural Network (can change number of hidden layers, nodes per hidden layer, etc)


filename = r'C:\Users\Alan Horst\MachineLearningNg\PythonML\data\BreastCancerdata.csv'
#filename = r'C:\Users\Alan Horst\MachineLearningNg\PythonML\data\ex1data2.csv'

In [29]:
import pandas as pd
import numpy as np
import math
import scipy
dataBC = pd.read_csv(filename)
dataBC.head()
#dataBC.plot(kind='scatter',x='radius_mean',y='smoothness_mean')

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [30]:
#Initialize machine learning algorithm of choice
Linear_Regression = 0 #do not use for classification, linear relationships

Logistic_Regression = 0 #only use for classification

Neural_Network = 1 #Use for classification


In [31]:
#Choose Data to Input
#DataInputx = ['area', 'occupants']
DataInputx = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
dataBC_X = dataBC[DataInputx].copy()
#Feature Scaling
for i in range(0,len(DataInputx)):
    mean = dataBC_X[DataInputx[i]].mean(axis=0)
    std = dataBC_X[DataInputx[i]].std(axis=0)
    def Norm(df):
        return (df[DataInputx[i]]-mean)/std
    dataBC_X[DataInputx[i]] = dataBC_X.apply(Norm, axis=1)
dataBC_X['zeroeth'] = 1

## Change this for new class data
if Logistic_Regression == 1 or Neural_Network == 1:
    BorM = {'M':1, 'B':0}
    dataBC.diagnosis = [BorM[item] for item in dataBC.diagnosis]

dataBC_Y = dataBC[['diagnosis']].copy()
Xtot = dataBC_X.values
X = Xtot[0:400,:]
ytot = dataBC_Y.values
y = ytot[0:400,:]
n = len(X[0,:])
m = len(y[:,0])
Xtest = Xtot[400:,:]
ytest = ytot[400:,:]

In [32]:
if Linear_Regression == 1:
    # Some gradient descent settings
    num_iters = 1500
    alpha = 0.01
    theta = np.zeros((n,1))
    J = 1/(2*m) * sum(np.square((X.dot(theta))-y))
        
    J_history = []
    iter = 0
    while iter < num_iters:
        temp = theta
        theta = temp - (alpha/(m) * sum(((X.dot(theta)-y).T).dot(X))).reshape((-1, 1))
        #np.append(J_history, computeCost(X, y, theta))
        iter += 1
    print('Theta Result:')
    print(theta)
    test = X[3]
    print('Algorithm Test Result:')
    print(np.dot(theta.T,test))

In [33]:
from scipy.optimize import minimize
regularization = 1 #To minimize effect of overfitting, set = 1
if Logistic_Regression == 1:
    iterations = 1500;
    alpha = 0.01;
    theta = np.zeros(n)
    initial_theta = np.zeros((n,1))
    def sigmoid(x):
        return 1/ (1 + np.exp(-x))
    def h(x, theta):
        return np.array(sigmoid(np.dot(x, theta)))
    sigmoid = np.vectorize(sigmoid)
    def costFunction(theta,x,y, lambda_):

        p1 = np.dot(y.T, np.log(h(x, theta)).reshape(m,1))
        p2 = np.dot((np.ones((m,1)) - y).T, np.log( 1 - h(x, theta)).reshape(m,1))
        summ = (p1 + p2)
        J = -summ[0]/m
        if regularization == 1:
            reg_J = lambda_/(2*m)*sum(np.square(theta[1:]))
            J[1:] = J[1:] + reg_J


        return J
    def gradientVect(theta, x, y, lambda_):
        gradient = []

        theta = theta.reshape(n,1)

        beta = h(x, theta) - y
        
        grad = np.dot(X.T, beta) * 1./m

        if regularization == 1:
            reg = theta[1:] * lambda_/m
            grad[1:] = grad[1:] + reg

        return grad.flatten()
        
    def optimizeTheta(x, y, nLabels, lambda_):

        for i in np.arange(0, nLabels):
            theta = np.zeros((n,1))
            res = minimize(costFunction, theta, args=(x, (y == i)*1, lambda_), method='BFGS',
                       jac=gradientVect, options={'maxiter':300})
            test = X[19]
            print(sigmoid(np.dot(res.x,test)))
        return res
    optimizeTheta(X,y,1,0.5)

In [34]:
import scipy
regularization = 1 #To minimize effect of overfitting, set = 1
if Neural_Network == 1:
    iterations = 500
    nodes = 10
    alpha = 0.01
    num_layers = 2
    epsilon_init = 0.12
    classes = len(np.unique(y))
    Y = np.zeros((classes,m))
    
    for i in range(0,m-1):
        ytemp = np.zeros((classes,1))
        position = [y[i]]
        ytemp[position] = 1
        Y[:,i] = ytemp[:,0]
        
    #weights negative??
    def randInitWeights(L_out,L_in):
        return np.random.rand(L_out,L_in)*2*epsilon_init-epsilon_init
    
    theta = []
    theta_init = []
    for i in range(0,num_layers+1):
        if i == 0:
            theta = [np.zeros((nodes,n))]
            timatx = randInitWeights(nodes,n)
            theta_init = [timatx]
        elif i == num_layers:
            tmatx = np.zeros((classes,nodes+1))
            theta.append(tmatx)
            timatx = randInitWeights(classes,nodes+1)
            theta_init.append(timatx)
        else:
            tmatx = np.zeros((nodes,nodes+1))
            theta.append(tmatx)
            timatx = randInitWeights(nodes,nodes+1)
            theta_init.append(timatx)

            
    def sigmoid(x):
        return 1/ (1 + np.exp(-x))
    sigmoid = np.vectorize(sigmoid)
    
    def sigmoid_grad(x):
        return np.dot(sigmoid(x), (1-sigmoid(x)))
    sigmoid_grad = np.vectorize(sigmoid_grad)
    
    def unroll(theta):
        for n in range(0,num_layers+1):
            if n == 0:
                unrolled = (theta[n].flatten())
            else:
                unrolled = np.append(unrolled,theta[n].flatten())
        return unrolled
    
    def fwd(theta,x,y,n,i): #forward propogation
        if n == 0:
            return np.insert(sigmoid(theta[n]@x[i,:].conj().T),0,1)
        elif n==num_layers:
            return sigmoid(theta[n]@fwd(theta,x,y,n-1,i))
        else:
            return np.insert(sigmoid(theta[n]@fwd(theta,x,y,n-1,i)),0,1)
        
    def costFunction(theta,x,y,lambda_):
        summ=0
        for i in range(0,m):
            p1 = np.dot(y[:,i], np.log(fwd(theta,x,y,num_layers,i)))
            p2 = np.dot((1 - y[:,i]),np.log(1 - fwd(theta,x,y,num_layers,i)))
            temp = (p1 + p2)
            summ += temp
        J = -summ/m
        if regularization == 1:
            sumreg = 0
            for i in range(1,num_layers):
                sumreg += sum(np.square(theta[i]))
            reg_J = lambda_/(2*m)*(sumreg)
            J = J + reg_J
        return J
    
    def bck(theta,x,y,n,i): #back propogation
        if n == 0:
            return (fwd(theta,x,y,num_layers,i) - y[:,i])
        elif n == num_layers:
            g_z = np.insert(sigmoid_grad(theta[0]@x[i,:]),0,1)
            d = theta[(num_layers+1)-n].conj().T@bck(theta,x,y,n-1,i)*g_z
            return d[1:]
        else:
            g_z = np.insert(sigmoid_grad(theta[(num_layers)-n]@fwd(theta,x,y,(num_layers-1)-n,i)),0,1)
            d = theta[(num_layers+1)-n].conj().T@bck(theta,x,y,n-1,i)*g_z
            return d[1:]    
        
    def gradientVect(theta, x, y, lambda_):
        gradient = []
        theta_grad = []
        reg_theta = []
        for j in range(0,num_layers+1):
            for i in range(0,m):        
                    if j == 0:
                        if i == 0:
                            theta_grad = [np.outer(bck(theta,x,y,num_layers,i), x[i,:].conj().T)]
                        else:
                            theta_grad[0] += np.outer(bck(theta,x,y,num_layers,i), x[i,:].conj().T)
                    else:
                        if i == 0:
                            theta_grad.append(np.outer(bck(theta,x,y,(num_layers)-(j),i), fwd(theta,x,y,j-1,i).conj().T))
                        else:
                            theta_grad[j] += np.outer(bck(theta,x,y,(num_layers)-(j),i), fwd(theta,x,y,j-1,i).conj().T)
        for i in range(0,num_layers+1):
            theta_grad[i] = 1/m*(theta_grad[i])
            if regularization == 1:
                if i == 0:
                    reg_theta = [lambda_/m*(theta[i])]
                    theta_grad[i][:,1:] += reg_theta[i][:,1:]
                else:
                    reg_theta.append(lambda_/m*(theta[i]))
                    theta_grad[i][:,1:] += reg_theta[i][:,1:]
        grad = theta_grad
        return grad
    
    theta_grad = gradientVect(theta_init, X, Y, 0.9)
    initial_nn_params = unroll(theta_init)

    def optimizeTheta(x, y, theta, alpha, num_iters, lambda_):
        J_history = []
        for i in range(num_iters):
            cost = costFunction(theta,x,y,lambda_)
            theta_grad = gradientVect(theta, x, y, lambda_)
            theta_fin = theta
            J_history.append(cost)
            for n in range(0,num_layers+1):
                    theta_fin[n] = theta_fin[n] - (alpha*theta_grad[n])
        return theta_fin , J_history
                
    nnTheta, nnJ_history = optimizeTheta(X,Y,theta_init,0.8,400,0.9)

In [49]:
    z = len(ytest[:,0])
    Ytest = np.zeros((classes,z))
    
    for i in range(0,z-1):
        ytemp = np.zeros((classes,1))
        position = [ytest[i]]
        ytemp[position] = 1
        Ytest[:,i] = ytemp[:,0]
        
    def predict(theta,X):
        h1 = sigmoid(X @ theta[0].conj().T)
        X0 = np.ones((z,1))
        h1 = np.hstack((X0,h1))
        h2 = sigmoid(h1 @ theta[1].conj().T)
        h2 = np.hstack((X0,h2))
        h3 = sigmoid(h2 @ theta[2].conj().T)
        return h3
    pred = predict(nnTheta,Xtest)
    actual = Ytest
    out = actual == np.around(pred.conj().T)
    fin = np.mean(1*out[0,:]) * 100
    print('Test Data Set Accuracy...')
    print(fin)

Test Data Set Accuracy...
96.44970414201184
